In [ ]:
from Bio import SeqIO
import numpy as np


In [ ]:
 from Bio.SeqIO.FastaIO import SimpleFastaParser

In [ ]:
FILES = ["cyto.fasta", "mito.fasta", "nucleus.fasta", "secreted.fasta"]

In [ ]:
def get_data(filename):
    with open("data/"+filename) as f:
        for values in SimpleFastaParser(f):
            print(values)